##### Copyright 2019 The TensorFlow Authors. | 2021 Philipp Kalytta

In [1]:
#@title Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
# https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

# Text Generierung mit einem Recurrent Neural Network


<table class="tfo-notebook-buttons" align="left">
  <td>
    <a target="_blank" href="https://www.tensorflow.org/tutorials/text/text_generation"><img src="https://www.tensorflow.org/images/tf_logo_32px.png" />View on TensorFlow.org</a>
  </td>
  <td>
    <a target="_blank" href="https://colab.research.google.com/github/tensorflow/docs/blob/master/site/en/tutorials/text/text_generation.ipynb"><img src="https://www.tensorflow.org/images/colab_logo_32px.png" />Run in Google Colab</a>
  </td>
  <td>
    <a target="_blank" href="https://github.com/tensorflow/docs/blob/master/site/en/tutorials/text/text_generation.ipynb"><img src="https://www.tensorflow.org/images/GitHub-Mark-32px.png" />View source on GitHub</a>
  </td>
  <td>
    <a href="https://storage.googleapis.com/tensorflow_docs/docs/site/en/tutorials/text/text_generation.ipynb"><img src="https://www.tensorflow.org/images/download_logo_32px.png" />Download notebook</a>
  </td>
</table>

Hier wird demonstriert, wie man mit einem zeichenbasierten RNN Text erzeugt. DU kannst hier mit einem Datensatz von Shakespeares Schriften aus dem Buch [The Unreasonable Effectiveness of Recurrent Neural Networks] von Andrej Karpathy (http://karpathy.github.io/2015/05/21/rnn-effectiveness/) arbeiten, oder deine eigene Textdatei verwenden. Geben Sie eine Sequenz von Zeichen aus diesen Daten ("Shakespeare") und trainieren Sie ein Modell, um das nächste Zeichen in der Sequenz ("e") vorherzusagen. Längere Textsequenzen können durch wiederholtes Aufrufen des Modells erzeugt werden.

Hinweis: Aktivieren Sie die GPU-Beschleunigung, um dieses Notizbuch schneller auszuführen. In Colab: *Laufzeit > Laufzeittyp ändern > Hardwarebeschleuniger > GPU*.

Dieses Colab-Book enthält lauffähigen Code, der mit [tf.keras] (https://www.tensorflow.org/guide/keras/sequential_model) und [eager execution] (https://www.tensorflow.org/guide/eager) implementiert wurde. Im Folgenden siehst du die Beispielausgabe, wenn das Modell in diesem Programm 30 Epochen lang trainiert und mit dem Zeichen "Q" gestartet wurde:

<pre>
QUEENE:
I had thought thou hadst a Roman; for the oracle,
Thus by All bids the man against the word,
Which are so weak of care, by old care done;
Your children were in your holy love,
And the precipitation through the bleeding throne.

BISHOP OF ELY:
Marry, and will, my lord, to weep in such a one were prettiest;
Yet now I was adopted heir
Of the world's lamentable day,
To watch the next way with his father with his face?

ESCALUS:
The cause why then we are all resolved more sons.

VOLUMNIA:
O, no, no, no, no, no, no, no, no, no, no, no, no, no, no, no, no, no, no, no, no, it is no sin it should be dead,
And love and pale as any will to that word.

QUEEN ELIZABETH:
But how long have I heard the soul for this world,
And show his hands of life be proved to stand.

PETRUCHIO:
I say he look'd on, if I must be content
To stay him from the fatal of our country's bliss.
His lordship pluck'd from this sentence then for prey,
And then let us twain, being the moon,
were she such a case as fills m
</pre>

Während einige der Sätze grammatikalisch korrekt sind, ergeben die meisten keinen Sinn. Das Modell hat nicht die Bedeutung der Wörter gelernt:

* Das Modell ist zeichenbasiert. Als das Training begann, wusste das Modell nicht, wie ein englisches Wort geschrieben wird oder dass Wörter überhaupt eine Texteinheit sind.

* Die Struktur der Ausgabe ähnelt einem Theaterstück - die Textblöcke beginnen in der Regel mit dem Namen des Sprechers, in Großbuchstaben, ähnlich wie im Datensatz.

* Wie unten gezeigt wird, wird das Modell auf kleinen Textmengen (je 100 Zeichen) trainiert und ist dennoch in der Lage, eine längere Textfolge mit kohärenter Struktur zu erzeugen.

## Setup

### Import TensorFlow and other libraries

In [2]:
import tensorflow as tf
from tensorflow.keras.layers.experimental import preprocessing

import numpy as np
import os
import time

In [3]:
import ssl
ssl._create_default_https_context = ssl._create_unverified_context

### Download the Shakespeare dataset

Ändere die folgende Zeile, wenn du deinen eigenen Datensatz verwenden willst. Dafür muss der Datensatz als Textdatei im Internet verfügbar sein.

Hier gibt es z.B. die Bibel: 
https://info1.sermon-online.com/german/MartinLuther-1912/Martin_Luther_Uebersetzung_1912.txt
https://www.sermon-online.de/search.pl?lang=de&id=6068

Oder hier Goethes Faust:
https://raw.githubusercontent.com/martinth/mobverdb/master/faust.txt

Star Wars Screenplay: http://www.scifiscripts.com/scripts/swd1_5-74.txt

Der Datensatz sollte möglichst groß sein, aber nicht zu groß.

In [4]:
#path_to_file = tf.keras.utils.get_file('shakespeare.txt', 'https://storage.googleapis.com/download.tensorflow.org/data/shakespeare.txt')
#path_to_file = tf.keras.utils.get_file("bibel.txt", "https://info1.sermon-online.com/german/MartinLuther-1912/Martin_Luther_Uebersetzung_1912.txt")
path_to_file = tf.keras.utils.get_file("goethe.txt", "https://raw.githubusercontent.com/martinth/mobverdb/master/faust.txt")


581632/568500 [==============================] - 0s 0us/step


### Read the data

Schauen wir uns den Text mal an. - Sollte das Format nicht utf-8 sein, muss hier natürlich ein anderes Format gewählt werden. z.B. cp1252 für Windows

In [5]:
# Read, then decode for py2 compat.
#text = open(path_to_file, 'rb').read().decode(encoding='cp1252')
text = open(path_to_file, 'rb').read().decode(encoding='utf-8')
# length of text is the number of characters in it
print(f'Length of text: {len(text)} characters')

Length of text: 560933 characters


In [6]:
# Take a look at the first 250 characters in text
print(text[:250])

The Project Gutenberg EBook of Faust: Der Tragödie erster Teil, by 
Johann Wolfgang von Goethe

This eBook is for the use of anyone anywhere at no cost and with
almost no restrictions whatsoever.  You may copy it, give it away or
re-use it under


In [7]:
# The unique characters in the file
vocab = sorted(set(text))
print(f'{len(vocab)} unique characters')

94 unique characters


## Process the text

### Vektorisierung des Textes

Vor dem Training müssen Sie die Strings in eine numerische Darstellung umwandeln. 

Die Schicht `preprocessing.StringLookup` kann jedes Zeichen in eine numerische ID umwandeln. Dazu muss der Text zunächst in Token zerlegt werden.

In [8]:
example_texts = ['abcdefg', 'xyz']

chars = tf.strings.unicode_split(example_texts, input_encoding='UTF-8')
chars

<tf.RaggedTensor [[b'a', b'b', b'c', b'd', b'e', b'f', b'g'], [b'x', b'y', b'z']]>

*Nun* erstellen Sie die Schicht "Preprocessing.StringLookup":

In [9]:
ids_from_chars = preprocessing.StringLookup(
    vocabulary=list(vocab))

*Es* konvertiert Form-Token in Zeichen-IDs, die mit `0` aufgefüllt werden:

In [10]:
ids = ids_from_chars(chars)
ids

<tf.RaggedTensor [[61, 62, 63, 64, 65, 66, 67], [84, 85, 86]]>

Da das Ziel dieser KI die Erzeugung von Text ist, wird es auch wichtig sein, diese Darstellung zu invertieren und daraus menschenlesbare Zeichenketten zu gewinnen. Hierfür können Sie `preprocessing.StringLookup(..., invert=True)` verwenden.  

Hinweis: Verwenden Sie hier statt der Übergabe des mit `sorted(set(text))` erzeugten Originalvokabulars die Methode `get_vocabulary()` der Schicht `preprocessing.StringLookup`, damit die Padding- und `[UNK]`-Token auf die gleiche Weise gesetzt werden.

In [11]:
chars_from_ids = tf.keras.layers.experimental.preprocessing.StringLookup(
    vocabulary=ids_from_chars.get_vocabulary(), invert=True)

Diese Schicht gewinnt die Zeichen aus den Vektoren der IDs und gibt sie als `tf.RaggedTensor` von Zeichen zurück:

In [12]:
chars = chars_from_ids(ids)
chars

<tf.RaggedTensor [[b'a', b'b', b'c', b'd', b'e', b'f', b'g'], [b'x', b'y', b'z']]>

Sie können `tf.strings.reduce_join` verwenden, um die Zeichen wieder zu Strings zu verbinden. 

In [13]:
tf.strings.reduce_join(chars, axis=-1).numpy()

array([b'abcdefg', b'xyz'], dtype=object)

In [14]:
def text_from_ids(ids):
  return tf.strings.reduce_join(chars_from_ids(ids), axis=-1)

### The prediction task

Was ist bei einem Zeichen oder einer Zeichenfolge das wahrscheinlichste nächste Zeichen? Dies ist die Aufgabe, für die Sie das Modell trainieren. Die Eingabe für das Modell wird eine Sequenz von Zeichen sein, und du trainierst das Modell, um die Ausgabe - das nächste Zeichen - bei jedem Zeitschritt vorherzusagen.

Da RNNs einen internen Zustand beibehalten, der von den zuvor gesehenen Elementen abhängt, ist die Frage: Was ist das nächste Zeichen, wenn alle bis zu diesem Zeitpunkt berechneten Zeichen vorliegen?


### Training Bespiele

Als nächstes wird der Text in Beispielsequenzen aufgeteilt. Jede Eingabesequenz enthält `seq_length` Zeichen aus dem Text.

Für jede Eingabesequenz enthalten die entsprechenden Ziele die gleiche Länge des Textes, nur um ein Zeichen nach rechts verschoben.

Zerlegen Sie also den Text in Abschnitte von `seq_length+1`. Nehmen wir zum Beispiel an, `seq_length` ist 4 und unser Text ist "Hallo". Die Eingabesequenz wäre "Hell", und die Zielsequenz "ello".

Verwenden Sie dazu zunächst die Funktion `tf.data.Dataset.from_tensor_slices`, um den Textvektor in einen Strom von Zeichenindizes zu konvertieren.

In [15]:
all_ids = ids_from_chars(tf.strings.unicode_split(text, 'UTF-8'))
all_ids

<tf.Tensor: shape=(560933,), dtype=int64, numpy=array([52, 68, 65, ..., 16,  2,  1])>

In [16]:
ids_dataset = tf.data.Dataset.from_tensor_slices(all_ids)

In [17]:
for ids in ids_dataset.take(10):
    print(chars_from_ids(ids).numpy().decode('utf-8'))

T
h
e
 
P
r
o
j
e
c


In [18]:
seq_length = 100
examples_per_epoch = len(text)//(seq_length+1)

Mit der "Batch"-Methode können Sie diese einzelnen Zeichen leicht in Sequenzen der gewünschten Größe umwandeln.

In [19]:
sequences = ids_dataset.batch(seq_length+1, drop_remainder=True)

for seq in sequences.take(1):
  print(chars_from_ids(seq))

tf.Tensor(
[b'T' b'h' b'e' b' ' b'P' b'r' b'o' b'j' b'e' b'c' b't' b' ' b'G' b'u'
 b't' b'e' b'n' b'b' b'e' b'r' b'g' b' ' b'E' b'B' b'o' b'o' b'k' b' '
 b'o' b'f' b' ' b'F' b'a' b'u' b's' b't' b':' b' ' b'D' b'e' b'r' b' '
 b'T' b'r' b'a' b'g' b'\xc3\xb6' b'd' b'i' b'e' b' ' b'e' b'r' b's' b't'
 b'e' b'r' b' ' b'T' b'e' b'i' b'l' b',' b' ' b'b' b'y' b' ' b'\r' b'\n'
 b'J' b'o' b'h' b'a' b'n' b'n' b' ' b'W' b'o' b'l' b'f' b'g' b'a' b'n'
 b'g' b' ' b'v' b'o' b'n' b' ' b'G' b'o' b'e' b't' b'h' b'e' b'\r' b'\n'
 b'\r' b'\n' b'T' b'h'], shape=(101,), dtype=string)


Es ist einfacher zu sehen, was dies bewirkt, wenn Sie die Token wieder zu Strings zusammenfügen:

In [20]:
for seq in sequences.take(5):
  print(text_from_ids(seq).numpy())

b'The Project Gutenberg EBook of Faust: Der Trag\xc3\xb6die erster Teil, by \r\nJohann Wolfgang von Goethe\r\n\r\nTh'
b'is eBook is for the use of anyone anywhere at no cost and with\r\nalmost no restrictions whatsoever.  Y'
b'ou may copy it, give it away or\r\nre-use it under the terms of the Project Gutenberg License included\r'
b'\nwith this eBook or online at www.gutenberg.net\r\n\r\n\r\nTitle: Faust: Der Trag\xc3\xb6die erster Teil\r\n\r\nAuthor'
b': Johann Wolfgang von Goethe\r\n\r\nPosting Date: January 26, 2010 [EBook #2229]\r\nRelease Date: June 2000'


Fürs Training benötigst Du einen Datensatz mit `(Eingabe, Bezeichnung)`-Paaren. Wobei `Eingabe` und 
Label" Sequenzen sind. Bei jedem Zeitschritt ist die Eingabe das aktuelle Zeichen und das Label ist das nächste Zeichen. 

Hier ist eine Funktion, die eine Sequenz als Eingabe nimmt, sie dupliziert und verschiebt, um die Eingabe und das Label für jeden Zeitschritt auszurichten:

In [21]:
def split_input_target(sequence):
    input_text = sequence[:-1]
    target_text = sequence[1:]
    return input_text, target_text

In [22]:
split_input_target(list("Tensorflow"))

(['T', 'e', 'n', 's', 'o', 'r', 'f', 'l', 'o'],
 ['e', 'n', 's', 'o', 'r', 'f', 'l', 'o', 'w'])

In [23]:
dataset = sequences.map(split_input_target)

In [24]:
for input_example, target_example in dataset.take(1):
    print("Input :", text_from_ids(input_example).numpy())
    print("Target:", text_from_ids(target_example).numpy())

Input : b'The Project Gutenberg EBook of Faust: Der Trag\xc3\xb6die erster Teil, by \r\nJohann Wolfgang von Goethe\r\n\r\nT'
Target: b'he Project Gutenberg EBook of Faust: Der Trag\xc3\xb6die erster Teil, by \r\nJohann Wolfgang von Goethe\r\n\r\nTh'


### Create training batches

Sie haben `tf.data` verwendet, um den Text in handhabbare Sequenzen aufzuteilen. Aber bevor Sie diese Daten in das Modell einspeisen, müssen Sie die Daten mischen und in Stapel packen.

In [25]:
# Batch size
BATCH_SIZE = 128

# Buffer size to shuffle the dataset
# (TF data is designed to work with possibly infinite sequences,
# so it doesn't attempt to shuffle the entire sequence in memory. Instead,
# it maintains a buffer in which it shuffles elements).
BUFFER_SIZE = 20000

dataset = (
    dataset
    .shuffle(BUFFER_SIZE)
    .batch(BATCH_SIZE, drop_remainder=True)
    .prefetch(tf.data.experimental.AUTOTUNE))

dataset

<PrefetchDataset element_spec=(TensorSpec(shape=(128, 100), dtype=tf.int64, name=None), TensorSpec(shape=(128, 100), dtype=tf.int64, name=None))>

## Build The Model

In diesem Abschnitt wird das Modell als Subklasse `keras.Model` definiert (Details siehe [Erstellen neuer Schichten und Modelle über Subklassen] (https://www.tensorflow.org/guide/keras/custom_layers_and_models)). 

Dieses Modell hat drei Schichten:

* `tf.keras.layers.Embedding`: Die Eingabeschicht. Eine trainierbare Nachschlagetabelle, die jede Zeichen-ID auf einen Vektor mit den Dimensionen `embedding_dim` abbildet;
* `tf.keras.layers.GRU`: Ein Typ von RNN mit der Größe `units=rnn_units` (Sie können hier auch eine LSTM-Schicht verwenden.)
* `tf.keras.layers.Dense`: Die Ausgabeschicht, mit Ausgaben der Größe `vocab_size`. Sie gibt für jedes Zeichen im Vokabular ein Logit aus. Dies sind die Log-Wahrscheinlichkeiten für jedes Zeichen gemäß dem Modell.

In [26]:
# Length of the vocabulary in chars
vocab_size = len(vocab)

# The embedding dimension
embedding_dim = 512

# Number of RNN units
rnn_units = 1024 #2048

In [27]:
class MyModel(tf.keras.Model):
  def __init__(self, vocab_size, embedding_dim, rnn_units):
    super().__init__(self)
    self.embedding = tf.keras.layers.Embedding(vocab_size, embedding_dim)
    self.gru = tf.keras.layers.GRU(rnn_units,
                                   return_sequences=True,
                                   return_state=True)
    self.dense = tf.keras.layers.Dense(vocab_size)

  def call(self, inputs, states=None, return_state=False, training=False):
    x = inputs
    x = self.embedding(x, training=training)
    if states is None:
      states = self.gru.get_initial_state(x)
    x, states = self.gru(x, initial_state=states, training=training)
    x = self.dense(x, training=training)

    if return_state:
      return x, states
    else:
      return x

In [28]:
model = MyModel(
    # Be sure the vocabulary size matches the `StringLookup` layers.
    vocab_size=len(ids_from_chars.get_vocabulary()),
    embedding_dim=embedding_dim,
    rnn_units=rnn_units)

Für jedes Zeichen sucht das Modell die Einbettung, führt die GRU einen Zeitschritt mit der Einbettung als Eingabe aus und wendet die dichte Schicht an, um Logits zu erzeugen, die die Log-Wahrscheinlichkeit des nächsten Zeichens vorhersagen:

![A drawing of the data passing through the model](https://github.com/tensorflow/docs/blob/master/site/en/tutorials/text/images/text_generation_training.png?raw=1)

Hinweis: Zum Training könnten Sie hier ein `keras.Sequential`-Modell verwenden. Um später Text zu generieren, müssen Sie den internen Zustand des RNNs verwalten. Es ist einfacher, die Eingabe- und Ausgabeoptionen für den Zustand im Voraus einzubinden, als die Modellarchitektur später umzustellen. Weitere Details finden Sie in der [Keras RNN-Anleitung](https://www.tensorflow.org/guide/keras/rnn#rnn_state_reuse).

## Try the model
Führe nun das Modell aus, um zu sehen, ob es sich wie erwartet verhält.

Prüfen Sie zunächst die Form der Ausgabe:

In [29]:
for input_example_batch, target_example_batch in dataset.take(1):
    example_batch_predictions = model(input_example_batch)
    print(example_batch_predictions.shape, "# (batch_size, sequence_length, vocab_size)")

(128, 100, 95) # (batch_size, sequence_length, vocab_size)


Im obigen Beispiel ist die Sequenzlänge der Eingabe "100", aber das Modell kann auf Eingaben beliebiger Länge ausgeführt werden:

In [30]:
model.summary()

Model: "my_model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       multiple                  48640     
                                                                 
 gru (GRU)                   multiple                  4724736   
                                                                 
 dense (Dense)               multiple                  97375     
                                                                 
Total params: 4,870,751
Trainable params: 4,870,751
Non-trainable params: 0
_________________________________________________________________


Um tatsächliche Vorhersagen aus dem Modell zu erhalten, müssen Sie eine Stichprobe aus der Ausgabeverteilung ziehen, um tatsächliche Zeichenindizes zu erhalten. Diese Verteilung ist durch die Logits über das Zeichenvokabular definiert.

Hinweis: Es ist wichtig, aus dieser Verteilung eine _Stichprobe_ zu nehmen, da das Modell durch die Verwendung des _argmax_ der Verteilung leicht in einer Schleife stecken bleiben kann.

Probieren Sie es für das erste Beispiel im Stapel aus:

In [31]:
sampled_indices = tf.random.categorical(example_batch_predictions[0], num_samples=1)
sampled_indices = tf.squeeze(sampled_indices, axis=-1).numpy()

Dadurch erhalten wir in jedem Zeitschritt eine Vorhersage des nächsten Zeichenindexes:

In [32]:
sampled_indices

array([31,  4, 34, 54, 40,  2, 58, 82, 29,  3, 54, 51, 40, 84,  8, 54, 55,
       50, 72, 39, 73, 37, 79, 60, 50, 56, 67, 91, 78, 14, 68, 68, 19, 40,
        5, 83, 13, 57,  7, 18, 35, 20, 55, 70, 60, 55,  1, 27, 20, 53, 51,
       21, 70, 48, 26, 63,  1, 65, 26, 32, 54, 89, 27, 37, 77, 73, 43, 65,
       85, 23, 23, 71, 17, 92, 78, 83, 42, 29, 51, 56, 20, 53, 91, 11,  4,
       91, 45,  1,  7,  9, 34, 46,  7, 87,  0, 77, 78, 82, 27, 47])

Dekodieren Sie diese, um den von diesem untrainierten Modell vorhergesagten Text zu sehen:

In [33]:
print("Input:\n", text_from_ids(input_example_batch[0]).numpy())
print()
print("Next Char Predictions:\n", text_from_ids(sampled_indices).numpy())

Input:
 b'et,\r\n  \xc3\xbcberfl\xc3\xbcssig, ewig helle\r\n  Rings durch alle Welten flie\xc3\x9fet--\r\n\r\n  MARIA AEGYPTIACA:\r\n  Bei de'

Next Char Predictions:
 b'?!BVH\rZv; VSHx%VWRlGmEs]RXg\xc3\xa4r,hh1H"w+Y$0C2Wj]W\n92US3jP8c\ne8@V\xc3\x9c9EqmKey55k/\xc3\xb6rwJ;SX2U\xc3\xa4)!\xc3\xa4M\n$\'BN$\xc3\x84[UNK]qrv9O'


## Train the model

An diesem Punkt kann das Problem als Standard-Klassifikationsproblem behandelt werden. In Anbetracht des vorherigen RNN-Zustands und der Eingabe in diesem Zeitschritt, sagen Sie die Klasse des nächsten Zeichens voraus.

### Attach an optimizer, and a loss function

Die Standardverlustfunktion `tf.keras.losses.sparse_categorical_crossentropy` funktioniert in diesem Fall, weil sie auf die letzte Dimension der Vorhersagen angewendet wird.

Da dein Modell Logits zurückgibt, müssen Sie das Flag `from_logits` setzen.


In [34]:
loss = tf.losses.SparseCategoricalCrossentropy(from_logits=True)

In [35]:
example_batch_loss = loss(target_example_batch, example_batch_predictions)
mean_loss = example_batch_loss.numpy().mean()
print("Prediction shape: ", example_batch_predictions.shape, " # (batch_size, sequence_length, vocab_size)")
print("Mean loss:        ", mean_loss)

Prediction shape:  (128, 100, 95)  # (batch_size, sequence_length, vocab_size)
Mean loss:         4.5542107


Ein neu initialisiertes Modell sollte sich seiner Sache nicht zu sicher sein, die ausgegebenen Logits sollten alle ähnliche Größenordnungen haben. Um dies zu bestätigen, können Sie prüfen, ob der Exponentialwert des mittleren Verlusts ungefähr gleich der Vokabulargröße ist. Ein viel höherer Verlust bedeutet, dass das Modell sich seiner falschen Antworten sicher ist und schlecht initialisiert wurde:

In [36]:
tf.exp(mean_loss).numpy()

95.03171

Konfigurieren Sie das Trainingsverfahren mit der Methode `tf.keras.Model.compile`. Verwenden Sie `tf.keras.optimizers.Adam` mit Standardargumenten und der Verlustfunktion.

In [37]:
model.compile(optimizer='adam', loss=loss)

### Configure checkpoints

Verwenden Sie ein `tf.keras.callbacks.ModelCheckpoint`, um sicherzustellen, dass Checkpoints während des Trainings gespeichert werden:

In [38]:
# Directory where the checkpoints will be saved
checkpoint_dir = './training_checkpoints'
# Name of the checkpoint files
checkpoint_prefix = os.path.join(checkpoint_dir, "ckpt_{epoch}")

checkpoint_callback = tf.keras.callbacks.ModelCheckpoint(
    filepath=checkpoint_prefix,
    save_weights_only=True)

### Execute the training



Um die Trainingszeit angemessen zu halten, verwenden Sie 10 Epochen zum Trainieren des Modells. Stellen Sie in Colab die Laufzeit auf GPU für schnelleres Training ein.

In [39]:
EPOCHS = 5

In [40]:
history = model.fit(dataset, epochs=EPOCHS, callbacks=[checkpoint_callback])

Epoch 1/5
43/43 [==============================] - 13s 225ms/step - loss: 3.4303
Epoch 2/5
43/43 [==============================] - 11s 226ms/step - loss: 2.3361
Epoch 3/5
43/43 [==============================] - 11s 225ms/step - loss: 2.1125
Epoch 4/5
43/43 [==============================] - 11s 226ms/step - loss: 1.9936
Epoch 5/5
43/43 [==============================] - 11s 229ms/step - loss: 1.8983


## Generate text

Der einfachste Weg, mit diesem Modell Text zu erzeugen, ist, es in einer Schleife laufen zu lassen und den internen Zustand des Modells während der Ausführung zu verfolgen.

![Um Text zu generieren, wird die Ausgabe des Modells an die Eingabe zurückgegeben](https://github.com/tensorflow/docs/blob/master/site/en/tutorials/text/images/text_generation_sampling.png?raw=1)

Jedes Mal, wenn Sie das Modell aufrufen, übergibst du etwas Text und einen internen Zustand. Das Modell gibt eine Vorhersage für das nächste Zeichen und seinen neuen Zustand zurück. Übergibst du die Vorhersage und den Zustand zurück, um die Texterzeugung fortzusetzen.

Im Folgenden wird eine einstufige Vorhersage getroffen:

In [41]:
class OneStep(tf.keras.Model):
  def __init__(self, model, chars_from_ids, ids_from_chars, temperature=1.0):
    super().__init__()
    self.temperature = temperature
    self.model = model
    self.chars_from_ids = chars_from_ids
    self.ids_from_chars = ids_from_chars

    # Create a mask to prevent "[UNK]" from being generated.
    skip_ids = self.ids_from_chars(['[UNK]'])[:, None]
    sparse_mask = tf.SparseTensor(
        # Put a -inf at each bad index.
        values=[-float('inf')]*len(skip_ids),
        indices=skip_ids,
        # Match the shape to the vocabulary
        dense_shape=[len(ids_from_chars.get_vocabulary())])
    self.prediction_mask = tf.sparse.to_dense(sparse_mask)

  @tf.function
  def generate_one_step(self, inputs, states=None):
    # Convert strings to token IDs.
    input_chars = tf.strings.unicode_split(inputs, 'UTF-8')
    input_ids = self.ids_from_chars(input_chars).to_tensor()

    # Run the model.
    # predicted_logits.shape is [batch, char, next_char_logits]
    predicted_logits, states = self.model(inputs=input_ids, states=states,
                                          return_state=True)
    # Only use the last prediction.
    predicted_logits = predicted_logits[:, -1, :]
    predicted_logits = predicted_logits/self.temperature
    # Apply the prediction mask: prevent "[UNK]" from being generated.
    predicted_logits = predicted_logits + self.prediction_mask

    # Sample the output logits to generate token IDs.
    predicted_ids = tf.random.categorical(predicted_logits, num_samples=1)
    predicted_ids = tf.squeeze(predicted_ids, axis=-1)

    # Convert from token ids to characters
    predicted_chars = self.chars_from_ids(predicted_ids)

    # Return the characters and model state.
    return predicted_chars, states

In [42]:
one_step_model = OneStep(model, chars_from_ids, ids_from_chars, 0.7)

Lass es in einer Schleife laufen, um einen Text zu erzeugen. Wenn du dir den generierten Text aniehst, wirst du sehen, dass das Modell weiß, wann es groß schreibt, Absätze bildet und ein zu deinem Text ähnliches Schreibvokabular imitiert. Mit der geringen Anzahl von Trainingsepochen hat es noch nicht gelernt, zusammenhängende Sätze zu bilden.

ROMEO: ist hier das startwort, natürlich kann hier was anderes gewählt werden.

In [43]:
start = time.time()
states = None
next_char = tf.constant(['ROMEO:']) # DAS IST DAS STARTWORT
result = [next_char]

for n in range(2000):
  next_char, states = one_step_model.generate_one_step(next_char, states=states)
  result.append(next_char)

result = tf.strings.join(result)
end = time.time()
print(result[0].numpy().decode('utf-8'), '\n\n' + '_'*80)
print('\nRun time:', end - start)

ROMEO:
  Dan welng und Werwiten Gebeut und dust die Sang ein Gegegänztest.

  FAUST:
  Ich lar im grettem Sthin aus und sich aus einn ander mich,
  So schören Hast du soglich einen Wehich graufein Gestioben,
  Wied und hie wieder sind und ihre anzu zu verlicht,
  Die Leist zu schort in an die Sand den Harden, chennemt, de nocht,
  Min nicht der fürnen witheibe geuschen wir sie sich und sohleinglich ster und Ratze der Sculeben geist inn sie sind Hert der Gestens andund dem Speine.

  FAUST:
  Hern gehnigest zu immer dochen, ist Ehn ihr dehn floct,
  Dein Feraff vor sin wirlichten weinen, Lacht lieht gereinel
  Doch warm seinen hände gaunen so ich sehnen, Hert nicht!
  Ich fürm und es geinde Herrs es aulemer, der Gesten Proje tert.

  FAUST:
  Den rich, auf in gewause, sich sin mich er der Gelle
  Won dee mom deh nicht gange schonten,
  Die Wand mer weingand, wein die Auch Zumer vort vonichen werben steten sein.
  Wie yin ein jeden sich Fraust.
  Was immer erthe min zu ind der Unde klich

Das Einfachste, was Du tun kannst, um die Ergebnisse zu verbessern, ist, es länger zu trainieren (versuchen Sie `EPOCHS = 30`).

Du kannst auch mit einer anderen Startzeichenfolge experimentieren, eine weitere RNN-Schicht hinzufügen, um die Genauigkeit des Modells zu verbessern, oder den Temeperaturparameter anpassen, um mehr oder weniger zufällige Vorhersagen zu erzeugen.

Wenn Sie möchten, dass das Modell Text *schneller* erzeugt, ist es am einfachsten, die Texterzeugung zu stapeln. Im folgenden Beispiel generiert das Modell 5 Ausgaben in etwa der gleichen Zeit, die es für die obige Generierung von 1 Ausgabe benötigte. 

In [44]:
start = time.time()
states = None
next_char = tf.constant(['ROMEO:'])
result = [next_char]

for n in range(1000):
  next_char, states = one_step_model.generate_one_step(next_char, states=states)
  result.append(next_char)

result = tf.strings.join(result)
end = time.time()
print(result[0].numpy().decode('utf-8'), '\n\n' + '_'*80)
print('\nRun time:', end - start)

ROMEO:
  Ender Mandich er auf der Schunnen, das Mengertu des Leicht, zurvollst von, woll man sich an dun auf er elten, die Geschenn;
  Due mand in die Under iptend har,
  Das werreit sich mer Gokeit der Projech Gewürkt,
  Sol' ich mir decht, nuch une Schäuten Gleich beriggender.

  MARGACBER:
  Die sin der Hein die Sterkt endert nicht, im Lunder sie richt vor int grüffen nan die Herder.
  Me!  Dert sein Euch und Gat dan ein, ein Sich noche nicht;
  Ihr der Kand sir zurennon der Bleiben lichen,
  Dar seinen sich gester prau der Festenze Schinen siete wott doch fand ander Wielem Lirden;
  Ferm dor woret in ich ein do nuhnen mindes chehnen,
  De PHojen, wir ich zunen, Schinken!
  Hars nicht ich nicht du schwar del Werest, das Rocht.
  Har sich mein Seiden, forsten seid einen;
  Ich bein ein Getommen, micht er simmen Wom!
  Dan eine Gläckte wie son die Meich ing an sor und vorendert nich am auch num das in warks an der Sich das, eis wirmeinigter,
  Ich allen Sterft und rieben Bl 

________

## Exportieren Sie den Generator

Dieses Einzelschrittmodell kann leicht [gespeichert und wiederhergestellt] werden (https://www.tensorflow.org/guide/saved_model), so dass Sie es überall verwenden können, wo ein `tf.saved_model` akzeptiert wird.

In [45]:
tf.saved_model.save(one_step_model, 'one_step')
one_step_reloaded = tf.saved_model.load('one_step')

INFO:tensorflow:Assets written to: one_step/assets


INFO:tensorflow:Assets written to: one_step/assets


In [46]:
states = None
next_char = tf.constant(['ROMEO:'])
result = [next_char]

for n in range(100):
  next_char, states = one_step_reloaded.generate_one_step(next_char, states=states)
  result.append(next_char)

print(tf.strings.join(result)[0].numpy().decode("utf-8"))

ROMEO:
  Wenwist tur trou sein in Glakt' wister affeine!
  Die Heies derer werder Zway dem Heiler Stie Z


## Fortgeschrittene: Angepasstes Training

Das obige Trainingsverfahren ist einfach, gibt Ihnen aber nicht viel Kontrolle.
Es verwendet Teacher-Forcing, das verhindert, dass schlechte Vorhersagen an das Modell zurückgegeben werden, so dass das Modell nie lernt, sich von Fehlern zu erholen.

Nachdem Sie nun gesehen haben, wie das Modell manuell ausgeführt wird, werden Sie als nächstes die Trainingsschleife implementieren. Damit haben Sie einen Ansatzpunkt, wenn Sie z. B. _Lehrplan-Lernen_ implementieren möchten, um die Ausgabe des Modells im offenen Regelkreis zu stabilisieren.

Der wichtigste Teil einer benutzerdefinierten Trainingsschleife ist die Funktion "train step".

Verwenden Sie `tf.GradientTape`, um die Gradienten zu verfolgen. Sie können mehr über diesen Ansatz erfahren, indem Sie die [eager execution guide](https://www.tensorflow.org/guide/eager) lesen.

Die grundlegende Vorgehensweise ist:

1. Führen Sie das Modell aus und berechnen Sie den Verlust unter einem `tf.GradientTape`.
2. Berechnen Sie die Aktualisierungen und wenden Sie sie mithilfe des Optimierers auf das Modell an.

In [47]:
class CustomTraining(MyModel):
  @tf.function
  def train_step(self, inputs):
      inputs, labels = inputs
      with tf.GradientTape() as tape:
          predictions = self(inputs, training=True)
          loss = self.loss(labels, predictions)
      grads = tape.gradient(loss, model.trainable_variables)
      self.optimizer.apply_gradients(zip(grads, model.trainable_variables))

      return {'loss': loss}

Die obige Implementierung der Methode `train_step` folgt den [Keras' `train_step`-Konventionen] (https://www.tensorflow.org/guide/keras/customizing_what_happens_in_fit). Dies ist optional, aber es erlaubt Ihnen, das Verhalten des Train-Schrittes zu ändern und trotzdem die Methoden `Model.compile` und `Model.fit` von Keras zu verwenden.

In [48]:
model = CustomTraining(
    vocab_size=len(ids_from_chars.get_vocabulary()),
    embedding_dim=embedding_dim,
    rnn_units=rnn_units)

In [49]:
model.compile(optimizer = tf.keras.optimizers.Adam(),
              loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True))

In [50]:
model.fit(dataset, epochs=1)

43/43 [==============================] - 13s 225ms/step - loss: 3.3959


Oder wenn Sie mehr Kontrolle benötigen, können Sie Ihre eigene, komplett benutzerdefinierte Trainingsschleife schreiben:

In [51]:
EPOCHS = 10

mean = tf.metrics.Mean()

for epoch in range(EPOCHS):
    start = time.time()

    mean.reset_states()
    for (batch_n, (inp, target)) in enumerate(dataset):
        logs = model.train_step([inp, target])
        mean.update_state(logs['loss'])

        if batch_n % 50 == 0:
            template = f"Epoch {epoch+1} Batch {batch_n} Loss {logs['loss']:.4f}"
            print(template)

    # saving (checkpoint) the model every 5 epochs
    if (epoch + 1) % 5 == 0:
        model.save_weights(checkpoint_prefix.format(epoch=epoch))

    print()
    print(f'Epoch {epoch+1} Loss: {mean.result().numpy():.4f}')
    print(f'Time taken for 1 epoch {time.time() - start:.2f} sec')
    print("_"*80)

model.save_weights(checkpoint_prefix.format(epoch=epoch))

Epoch 1 Batch 0 Loss 2.5567

Epoch 1 Loss: 2.3271
Time taken for 1 epoch 11.22 sec
________________________________________________________________________________
Epoch 2 Batch 0 Loss 2.2147

Epoch 2 Loss: 2.1021
Time taken for 1 epoch 10.40 sec
________________________________________________________________________________
Epoch 3 Batch 0 Loss 2.0228

Epoch 3 Loss: 1.9842
Time taken for 1 epoch 10.37 sec
________________________________________________________________________________
Epoch 4 Batch 0 Loss 1.9219

Epoch 4 Loss: 1.8868
Time taken for 1 epoch 10.38 sec
________________________________________________________________________________
Epoch 5 Batch 0 Loss 1.8702

Epoch 5 Loss: 1.7999
Time taken for 1 epoch 10.57 sec
________________________________________________________________________________
Epoch 6 Batch 0 Loss 1.7252

Epoch 6 Loss: 1.7183
Time taken for 1 epoch 10.39 sec
________________________________________________________________________________
Epoch 7 Batch 0 